In [1]:
import dachi
import typing
import pydantic
from dachi.proc import F
from dachi.act import threaded_task
from dachi.store import Context
from dachi_tutorials.teach.base import OpenAILLM
from collections import deque
import dachi.asst.openai_asst
from dachi.asst.openai_asst import ParsedConv




In [2]:

class Blackboard(dachi.store.Blackboard):
    
    queued_synopses: deque = pydantic.Field(default_factory=deque)
    synopses: typing.Dict[str, str] = pydantic.Field(default_factory=dict)
    evaluations: typing.Dict[str, str] = pydantic.Field(default_factory=dict)
    background: str = pydantic.Field(default='')
    goal: str = pydantic.Field(default='')

blackboard = Blackboard()

In [ ]:
brainstorm = dachi.store.Shared()
op = dachi.asst.Op(
    OpenAILLM('gpt-4o-mini', [
        dachi.asst.adapt.xopenai.TextConv('content')]
    ), dachi.asst.ToText(role='user'), 
    out='content'
)
ctx = dachi.store.ContextStorage()

In [4]:

role = """
You are an excellent team lead for a movie synopsis writing team
who prides himself on writing top movie synopses.
"""

context = f"""

# Team Goal
Write a great story
"""

task = threaded_task(F(op, f"""
    {role}

    {context}

    You need to write a movie synopsis

""", _out=str), ctx.brainstorm, brainstorm)

In [5]:
task()

<TaskStatus.RUNNING: 'running'>

In [6]:
task()

<TaskStatus.RUNNING: 'running'>

In [7]:
task()

<TaskStatus.SUCCESS: 'success'>

In [9]:
brainstorm.data
#  print(ctx.brainstorm['error'])

'**Movie Title: "Echoes of Tomorrow"**\n\n**Genre:** Sci-Fi/Drama\n\n**Synopsis:**\n\nIn a near-future world ravaged by climate change and societal collapse, humanity teeters on the brink of extinction. Amidst this chaos, a brilliant but disillusioned quantum physicist, Dr. Elena Hart, stumbles upon a groundbreaking discovery: a technology that allows individuals to send their consciousness back in time for brief moments, providing them the chance to alter critical decisions and reshape their future.\n\nHaunted by guilt over her estranged father’s death during the early years of the crisis, Elena becomes fixated on using her invention to change her past. As she embarks on this deeply personal journey, she inadvertently attracts the attention of a shadowy organization that seeks to weaponize her technology for their own gain. Alongside her estranged childhood friend, Marcus, a resourceful activist determined to save the future, they must navigate their tumultuous emotions and face the e